In [1]:
!pip install datasets

In [2]:
!pip install tensorflow scikit-learn numpy pandas matplotlib


In [3]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
import random
import pandas as pd
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
!pip install transformers datasets torch accelerate


In [8]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer

# Load dataset
file_path = "/content/drive/MyDrive/mental_health_chatbot_dataset_with_emojis_classified.csv"
df = pd.read_csv(file_path)

# Check dataset columns
print("Dataset Columns:", df.columns)

# Create formatted text (combining User Input and Bot Initial Response)
df["formatted_text"] = df["User Input"] + " [SEP] " + df["Bot Initial Response"]

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

# Tokenize and pad sequences
tokenized_texts = df["formatted_text"].apply(lambda x: tokenizer.encode(x, truncation=True, padding="max_length", max_length=128))

# Convert to tensor format
input_ids = torch.tensor(tokenized_texts.to_list())

# Check shape
print("Tokenized input shape:", input_ids.shape)
# check image
# Input.ids format tensor sequences tokenizer padding token


Dataset Columns: Index(['User Input', 'Bot Initial Response', 'Follow-up Response (Yes)',
       'Follow-up Response (No)', 'Emotion'],
      dtype='object')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Tokenized input shape: torch.Size([98, 128])


In [9]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from torch.utils.data import Dataset

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Resize token embeddings to account for padding token
model.resize_token_embeddings(len(tokenizer))

# Custom dataset class
class ChatbotDataset(Dataset):
    def __init__(self, input_ids):
        self.input_ids = input_ids

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {"input_ids": self.input_ids[idx], "attention_mask": (self.input_ids[idx] != tokenizer.pad_token_id).long()}

# Create dataset
dataset = ChatbotDataset(input_ids)

# Data collator for dynamic padding
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-mental-health-chatbot",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    num_train_epochs=5,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    evaluation_strategy="no",
    do_train=True
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)

# Start fine-tuning
trainer.train()


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cse-210840131002 (cse-210840131002-other) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,1.215000
100,0.290100


TrainOutput(global_step=125, training_loss=0.6460423278808594, metrics={'train_runtime': 76.1938, 'train_samples_per_second': 6.431, 'train_steps_per_second': 1.641, 'total_flos': 32008273920000.0, 'train_loss': 0.6460423278808594, 'epoch': 5.0})

In [10]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)  # Check which device is being used

# Move model to device
model.to(device)


Using device: cuda


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [11]:
import random
import torch

# Expanded list of calming activities with instructions
calming_activities = {
    "guided meditation": "Find a quiet place, close your eyes, and take slow, deep breaths. Focus on your breathing or listen to a meditation guide. Here are some guided meditation links:\n"
                          "🧘 [YouTube - 5 Min Meditation](https://www.youtube.com/watch?v=inpok4MKVLM)\n"
                          "🧘 [Headspace - Basics Meditation](https://www.youtube.com/watch?v=eEk8HffjiW8)\n"
                          "🧘 [Calm App - Guided Meditation](https://www.calm.com/)",
    "deep breathing exercises": "Try the 4-7-8 technique: Inhale for 4 seconds, hold for 7 seconds, and exhale for 8 seconds. Repeat a few times.",
    "journaling": "Write down your thoughts in a journal. Expressing emotions on paper can help clear your mind.",
    "listening to calming music": "Play some soft, soothing music or nature sounds. Let the sounds relax your mind and body. Here are some links to calming music:\n"
                                  "🎵 [YouTube - Relaxing Music](https://www.youtube.com/watch?v=2OEL4P1Rz04)\n"
                                  "🎵 [Spotify - Calm Vibes](https://open.spotify.com/playlist/37i9dQZF1DX3Ogo9pFvBkY)\n"
                                  "🎵 [Nature Sounds](https://www.youtube.com/watch?v=eKFTSSKCzWA)",
    "going for a short walk": "Take a slow walk outside. Focus on your surroundings—the sounds, the air, and how your body feels as you move.",
    "progressive muscle relaxation": "Tense each muscle group for 5 seconds, then release. Start from your toes and work up to your head.",
    "visualization techniques": "Close your eyes and imagine a peaceful place. Picture the colors, sounds, and sensations to immerse yourself in it.",
    "talking to a trusted friend": "Reach out to someone you trust and share how you're feeling. Talking can lighten your emotional load.",
    "stretching exercises": "Gently stretch your arms, legs, and back. Stretching can help relieve tension and improve circulation.",
    "sipping a warm drink": "Make yourself a cup of herbal tea or warm milk. Slowly sip it while focusing on its taste and warmth.",
    "aromatherapy": "Use essential oils like lavender or chamomile to create a relaxing atmosphere. Inhale deeply and enjoy the soothing scents.",
    "reading a book": "Pick up a book or read something uplifting. Escaping into a good story can help shift your focus away from stress.",
    "doodling or coloring": "Try drawing, doodling, or using a coloring book. Engaging in creative activities can be very calming.",
    "taking a warm bath or shower": "A warm bath or shower can help relax your muscles and ease stress.",
    "practicing gratitude": "Write down three things you are grateful for today. Focusing on positive things can improve your mood.",
    "watching a comforting movie or show": "Watch a lighthearted or nostalgic movie or series to lift your spirits.",
    "playing with a pet": "Spend time with your pet, cuddle, or take them for a walk. Animals can provide comfort and companionship.",
    "gardening or spending time in nature": "Spend time tending to plants, watering them, or just sitting in a garden. Nature can be very grounding.",
    "yoga or gentle movement": "Do some light yoga or gentle stretching. Moving your body mindfully can relieve stress.",
    "doing a simple act of kindness": "Send a kind message to a friend, compliment someone, or do a small good deed to spread positivity."
}

# User history
user_history = {"last_suggestion": None}

# Casual responses
casual_responses = {
    "thank you": "You're welcome! 😊",
    "thanks": "No problem! 😊",
    "okay": "Got it! 💙",
    "alright": "Alright! 💙",
    "can you suggest another activity?": "I hear you. Let me find another activity! " + random.choice(list(calming_activities.keys()))
}

def suggest_activity():
    activity = random.choice(list(calming_activities.keys()))
    user_history["last_suggestion"] = activity
    return f"Would you like to try {activity}?"

def generate_response(user_input):
    input_text = user_input + " [SEP]"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    output = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id, do_sample=True, top_k=50, temperature=0.7)
    response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

    activity_suggestion = suggest_activity()
    return f"{response} {activity_suggestion}"

def get_alternative_suggestion():
    if user_history["last_suggestion"]:
        previous_suggestion = user_history["last_suggestion"]
        new_options = [activity for activity in calming_activities if activity != previous_suggestion]
        if new_options:
            new_suggestion = random.choice(new_options)
            user_history["last_suggestion"] = new_suggestion
            return f"Would you like to try {new_suggestion} instead?"
    return "I understand. Let me know how I can help."

def provide_guidance():
    if user_history["last_suggestion"]:
        activity = user_history["last_suggestion"]
        return f"Great choice! Here's how you can do {activity}: {calming_activities[activity]}"
    return "I'm glad to help! Let me know what you need."

# Chatbot loop
while True:
    user_input = input("You: ").strip().lower()
    if user_input in ["exit", "quit"]:
        break
    elif user_input in casual_responses:
        response = casual_responses[user_input]
    elif user_input in ["no", "not really", "i don't think so"]:
        response = get_alternative_suggestion()
    elif user_input in ["yes", "sure", "okay", "alright"]:
        response = provide_guidance()
    else:
        response = generate_response(user_input)
    print(f"Bot: {response}")


You: I am anxious


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Bot:  That sounds tough. Would you like to try a short guided meditation? 💡�️� 💙💙💙� 🌿 🌿🙌🌿🌿 ✨🌿‌�️‌🌿‌�t 🌿‌�‌t 🌿‌�‌‌️‌🌿‌‌� Would you like to try stretching exercises?
You: no
Bot: Would you like to try talking to a trusted friend instead?
You: no
Bot: Would you like to try stretching exercises instead?
You: yes
Bot: Great choice! Here's how you can do stretching exercises: Gently stretch your arms, legs, and back. Stretching can help relieve tension and improve circulation.
You: thank you
Bot: You're welcome! 😊
You: exit


In [12]:
# Define save path
save_path = "./gpt2_mental_health_chatbot"

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model saved to {save_path}")


Model saved to ./gpt2_mental_health_chatbot


In [13]:
import shutil

shutil.make_archive("gpt2_mental_health_chatbot", 'zip', save_path)
print("Model zipped successfully!")


Model zipped successfully!


In [14]:
from google.colab import files
files.download("gpt2_mental_health_chatbot.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>